In [7]:
# score coherence
from sentence_transformers import SentenceTransformer, util

GLOSS = {
    "business": ["đặt phòng khám", "tái khám", "hẹn lịch khám", "đo kính mắt", "mua bán"],
    "expertise": ["thị giác", "nhược thị", "tật khúc xạ", "bệnh về mắt", "khám mắt cho trẻ em", "bác sĩ mắt"],
}

def score_coh(sentence, model):
    s_embd = model.encode(sentence)
    domains = list(GLOSS.keys())
    
    score = 0
    
    for k in domains:
        for term in GLOSS[k]: 
            w_embd = model.encode(term)
            score += util.cos_sim(w_embd, s_embd)
    
    return score    

In [12]:
model_semantic = SentenceTransformer("./vietnamese-sbert/")
score_coh("Em có con nhỏ đang bị cận muốn cho đi khám bác ạ được không ạ", model_semantic)

tensor([[3.0575]])

In [13]:
# score interest
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

def score_interest(sentence, model, tokenizer):
    input_ids = torch.tensor([tokenizer.encode(sentence)])
    with torch.no_grad():
        out = model(input_ids)
        # print(out.logits.softmax(dim=-1).tolist())
    return out.logits.softmax(dim=-1).tolist() 

# Just like PhoBERT: INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!


In [14]:
model = RobertaForSequenceClassification.from_pretrained("./phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("./phobert-base-vietnamese-sentiment", use_fast=False)

score_interest("được đấy", model, tokenizer)

[[0.007860301062464714, 0.8275406956672668, 0.16459901630878448]]

In [18]:
import pandas as pd
df = pd.read_excel("../datasets/conversation_1.xlsx")
df

,Sentence,response_time
0,"ko phải con mình , mà xem còn thấy đau như vậy...",10
1,per nghe đi rồi khóc 1 trận cho thoải mái . đừ...,961
2,công nhận sáng tạo thật đấy,9
3,đòn tấn công cực gắt và cục sút của anh 😂😂😂,69
4,trời nắng nóng thế này mình muốn bán nước khôn...,9604
5,minh biết nữa ne,90
6,mấy thằng củ lol việt nam nhảm nhí :)),5
7,tui thi ́ ch va ̉ i lă ́ mày ma ̀ ăn nhỉ ̀ u n...,76
8,"bếp dầu , nhiều nhà vẫn dùng",1225


In [24]:
import pandas as pd
df2 = pd.read_excel("../datasets/conversation_7.xlsx")
df2

,Sentence,response_time
0,mình thi xong còn không khóc được 😢,11
1,"mỗi ngày mình ôm nhau 1 cái như này hén , hí hí",28
2,"per trừ điểm , trừ điểm",80
3,"5 triệu , phạt một con mụ lừa đảo , phá hủy ni...",1
4,cả tuần không được nói chuyện,900
5,per mày cẩn thận tao giết,28
6,cạn lời với bọn súc vật này làm xấu mặt người ...,4
7,bài này tao có 2 cảm xúc . đầu tiên về quán tr...,74


In [19]:
df.iloc[2,1]

9

In [20]:
import numpy as np
def proc_sentiment(sent):
    if np.argmax(sent) == 0:
        return abs(sent[1] - sent[2])
    elif np.argmax(sent) == 1:
        return sent[1]
    else:
        return -sent[2]

def conversation_value(data):
    model_semantic = SentenceTransformer("./vietnamese-sbert/")
    model = RobertaForSequenceClassification.from_pretrained("./phobert-base-vietnamese-sentiment")
    tokenizer = AutoTokenizer.from_pretrained("./phobert-base-vietnamese-sentiment", use_fast=False)
    
    val = 0
    for i, sentence in enumerate(list(data.Sentence)):
        val += score_coh(sentence, model_semantic)
        sent = score_interest(sentence, model, tokenizer)
        if i == 0: val += proc_sentiment(sent[0])
        else: val += proc_sentiment(sent[0]) / data.iloc[i, 1]
    
    return val

In [21]:
conversation_value(df)

tensor([[11.0799]])

In [25]:
conversation_value(df2)

tensor([[7.4449]])

In [31]:
df_test = pd.DataFrame({
    "Sentence":["Chào bác sĩ", "vâng ah", "cháu nhà em 8 tuổi đang bị mờ một bên mắt", "vậy em đưa cháu khám lúc 8h nhé", "dạ vâng em cảm ơn bác sĩ", "à em hỏi thêm tí ạ", "bao giờ thì mình có đơn thuốc"],
    "response_time":[1,2,4,3,2,3,4]
})

In [32]:
df_test

,Sentence,response_time
0,Chào bác sĩ,1
1,vâng ah,2
2,cháu nhà em 8 tuổi đang bị mờ một bên mắt,4
3,vậy em đưa cháu khám lúc 8h nhé,3
4,dạ vâng em cảm ơn bác sĩ,2
5,à em hỏi thêm tí ạ,3
6,bao giờ thì mình có đơn thuốc,4


In [33]:
conversation_value(df_test)

tensor([[14.4940]])